In [76]:
import pandas as pd
import numpy as np

In [77]:
import zipfile
import tensorflow as tf

In [78]:
seq_len = None

In [79]:
def remove_outliers(data, threshold=73):
    titles = {i: i.lower() for i in data.columns}
    return (data.rename(columns=titles).query('expected <= @threshold').copy())

In [80]:
def seq_pad(data, size, steps=seq_len):
    arr = []
    for i in data:
        _ = i.copy()
        _.resize(steps, size)
        arr.append(_)
    arr = np.array(arr)
    return arr

# Data Loading

training - 20 days of each month
test - remaining days
Both sets are independent
Calendar time and location information omitted:
    impossible to construct a local validation holdout subset independent from the rest of the training set
    no way to ensure that any two gauge readings are not correlated in time/space

In [81]:
#zf1 = zipfile.ZipFile('../../train.zip')
#zf2 = zipfile.ZipFile('../../test.zip')
#train = pd.read_csv(zf1.open(zipfile.ZipFile.namelist(zf1)[0]))
#test = pd.read_csv(zf2.open(zipfile.ZipFile.namelist(zf2)[0]))
train = pd.read_csv('../../train.csv', nrows=10000).pipe(remove_outliers)
test = pd.read_csv('../../test.csv', nrows=1000)

In [82]:
train.head()

,id,minutes_past,radardist_km,ref,ref_5x5_10th,ref_5x5_50th,ref_5x5_90th,refcomposite,refcomposite_5x5_10th,refcomposite_5x5_50th,...,rhohv_5x5_90th,zdr,zdr_5x5_10th,zdr_5x5_50th,zdr_5x5_90th,kdp,kdp_5x5_10th,kdp_5x5_50th,kdp_5x5_90th,expected
0,1,3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
1,1,16,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
2,1,25,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
3,1,35,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
4,1,45,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254


In [83]:
train_t = train.copy()

5-layer deep stacked Bi-RNN, 64-256 hidden units, dense after each hidden stack, 
last layer at each time position vector is fed into a dense layer with 1 output and ReLU. Take mean of the predictions from the entire top layer


# Preprocessing
exclude the rain gauges with readings above 70mm.
replace missing radar feature values with zero
each timestamp as a component in the feature vector, sequential nature of the input is preserved

In [84]:
#train_t['count_id'] = train_t.groupby('Id')['Id'].transform('count')
#train_t['count_None_Ref'] = train_t[~np.isnan(train_t.Ref)].transform('count')
train_nones = train_t[~np.isnan(train_t['ref'])]['id'].unique()
train_full = train_t['id']

In [85]:
train_cleaned = train[np.in1d(train_full, train_nones)].fillna(0.0).reset_index(drop=True)

In [86]:
ids = train_cleaned['id'].unique()
train_ids = np.random.choice(ids,
                               size=np.ceil(len(ids) * (9/10.)).astype(int),
                               replace=False)

In [95]:
train_data = train.loc[train['id'].isin(train_ids)]
test_data = train.loc[~train['id'].isin(train_ids)]
#[train.columns != 'expected']

In [99]:
train_data.head()

,id,minutes_past,radardist_km,ref,ref_5x5_10th,ref_5x5_50th,ref_5x5_90th,refcomposite,refcomposite_5x5_10th,refcomposite_5x5_50th,...,rhohv_5x5_90th,zdr,zdr_5x5_10th,zdr_5x5_50th,zdr_5x5_90th,kdp,kdp_5x5_10th,kdp_5x5_50th,kdp_5x5_90th,expected
30,4,0,9.0,18.5,16.0,19.0,23.5,24.5,22.0,23.5,...,0.998333,NaN,NaN,NaN,1.4375,NaN,NaN,NaN,NaN,4.064002
31,4,5,9.0,22.0,14.0,21.0,30.0,25.5,22.5,25.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.064002
32,4,9,9.0,35.5,24.5,30.5,35.5,35.5,25.5,31.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.064002
33,4,14,9.0,16.5,NaN,19.0,34.5,23.5,16.5,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.064002
34,4,19,9.0,16.0,NaN,11.0,18.0,20.5,13.5,18.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.064002


In [101]:
y_train = train_data.groupby('id')['expected'].last().values

y_test = test_data.groupby('id')['expected'].last().values

In [102]:
train_data.shape

(5702, 24)

In [103]:
y_train.shape

(484,)

In [107]:
train_data_arr = y_train.astype('float')
test_data_arr = y_test.astype('float')

seq_lens_train = train_data.groupby('id').size().values
seq_lens_test = test_data.groupby('id').size().values

In [109]:
train_data_arr.shape

(484,)

In [110]:
n_in = train_data_arr.shape[1]
n_out = 1
seq_len = max([seq_lens_train.max(), seq_lens_test.max()])
neurons = [128, 256]

IndexError: tuple index out of range

In [26]:
test = seq_pad(test_data_arr, n_in, seq_len)

### Model

In [27]:
tf.reset_default_graph()

In [28]:
X = tf.placeholder(dtype=tf.float32, shape=[None, seq_len , n_in])
y = tf.placeholder(tf.float32, [None])
seq_length = tf.placeholder(dtype=tf.int16, shape=[None])

with tf.variable_scope('rnn', initializer=tf.contrib.layers.variance_scaling_initializer()):
    rnn_cell_1 = tf.contrib.rnn.LSTMCell(num_units=neurons[0])
    rnn_cell_2 = tf.contrib.rnn.LSTMCell(num_units=neurons[1])
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell([rnn_cell_1, rnn_cell_2])
    outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32,
                                        sequence_length=seq_length)
    drop = tf.layers.dropout(inputs=states[1], rate=0.4)
    y_pred = tf.layers.dense(drop, n_out)

In [33]:
learning_rate = 0.05

error = (y-y_pred)
loss = tf.reduce_mean(tf.square(error), name="loss")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

mae = tf.reduce_mean(tf.abs(error))

In [34]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [46]:
n_samples = train_data_arr.shape[0]
n_epochs = 5
batch_size = len(train_data_arr) / 25
#printbatch_size


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        rand_idx = np.random.permutation(np.arange(n_samples))
        for batch_i in range(int(n_samples // batch_size)):
            batch_idx = rand_idx[int(batch_i*batch_size):int((batch_i+1)*batch_size)]
            X_batch = seq_pad(train_data_arr[batch_idx], n_in, seq_len)
            y_batch = y_train[batch_idx]
            len_batch = seq_lens_train[batch_idx]
            loss_val, _ = sess.run(
                [loss, training_op],
                feed_dict={X: X_batch, seq_length: len_batch, y: y_batch})
        mae_train = mae.eval(feed_dict={X: X_batch, seq_length: len_batch, y: y_batch})
        mae_test = mae.eval(feed_dict={X: test_pad, seq_length: seq_lens_test, y: y_test})
        print("{:4d}  Train loss: {:.4f}, Train MAE: {:.2f}  Test MAE: {:.2f}".format(
            epoch, loss_val, mae_train, mae_test))
        saver.save(sess, "my_reber_classifier")
        final_pred = y_pred.eval({X: test_pad, seq_length: seq_lens_test, y: y_test})

IndexError: index 2731 is out of bounds for axis 1 with size 484